In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("E:/Work/Varsity/research/paper3/kaggleTransactionPrediction/santander-customer-transaction-prediction/train.csv")

In [3]:
ds = pd.read_csv("E:/Work/Varsity/research/paper3/kaggleTransactionPrediction/santander-customer-transaction-prediction/test.csv")

In [4]:
df.shape

(200000, 202)

In [5]:
df.iloc[:,2:].columns

Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)

In [6]:
#dropping dupicate values in data
df.drop_duplicates(subset=df.iloc[:,2:].columns, inplace = True)

In [7]:
df.shape

(200000, 202)

In [8]:
df = df[~pd.isnull(df).any(axis=1)]

In [9]:
df.shape

(200000, 202)

In [10]:
df.iloc[:,2].unique().shape

(94672,)

In [11]:
def zscr(df):
    return (df-df.mean())/df.std()

In [12]:
for i in df.iloc[:,2:].columns:
    df[i] = zscr(df[i])

In [13]:
for i in ds.iloc[:,1:].columns:
    ds[i] = zscr(ds[i])

In [14]:
ds.iloc[:,1:].head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0.133981,2.327440,0.852788,1.286577,0.220421,0.339301,0.502073,0.507626,0.556963,1.007084,...,-1.174486,1.451556,-2.267582,-0.218044,-1.364610,1.818981,0.380898,1.962823,-0.131944,-0.526356
1,-0.700868,0.712421,0.226755,-0.780628,-1.162419,0.132014,0.699023,0.613949,-1.407053,-1.295196,...,1.631805,0.455056,-0.665991,1.703420,-0.770689,0.427339,-0.693327,1.041337,1.083557,-1.705006
2,-1.704485,-2.161573,-0.215177,0.126508,-0.503323,1.887814,-0.601564,1.087627,0.373845,0.628959,...,-0.865295,1.168598,0.036728,-0.285818,-1.597014,1.582004,-1.725836,-2.009577,1.338250,-1.916900
3,-0.698563,0.074754,0.499090,-0.103917,-1.379932,1.045197,-0.549869,1.178823,0.929498,-0.090600,...,1.401911,0.534952,-0.181048,0.064892,-0.894581,2.307692,0.305857,0.344305,-0.948040,-0.092749
4,0.344801,0.369147,1.299238,0.468834,-1.220509,-0.449118,1.670359,-1.729997,0.813549,-0.345580,...,0.227660,0.566586,-0.434094,0.013961,0.495831,-0.106558,-1.368275,-1.763732,-0.645851,-0.571064


In [15]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(df.iloc[:,2:], df.iloc[:,1], test_size = 0.2, random_state = 0)

In [16]:
def getrows(chunk):
    x = xTrain.iloc[chunk,:]
    y = yTrain.iloc[chunk]
    return x,y

In [17]:
from sklearn import linear_model
 
def iter_minibatches(chunksize):
    # Provide chunks one by one
    chunkstartmarker = 0
    while chunkstartmarker < xTrain.shape[0]:
        chunkrows = range(chunkstartmarker,chunkstartmarker+chunksize)
        X_chunk, y_chunk = getrows(chunkrows)
        yield X_chunk, y_chunk
        chunkstartmarker += chunksize
 
batcherator = iter_minibatches(chunksize=20000)
model = linear_model.SGDClassifier(penalty="l2")
#model = LogisticRegression()
 
# Train model
for X_chunk, y_chunk in batcherator:
    model.partial_fit(X_chunk, y_chunk, classes=np.unique(yTrain))
 
# Now make predictions with trained model
y_predicted = model.predict(xTest)


c:\users\ratul\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [18]:
from  sklearn.metrics  import accuracy_score
print(accuracy_score(yTest,y_predicted))

0.88025


In [19]:
xTrain = df.iloc[:,2:]
yTrain = df.iloc[:,1]
xTest = ds.iloc[:,1:]

In [20]:
from sklearn import linear_model
 
def iter_minibatches(chunksize):
    # Provide chunks one by one
    chunkstartmarker = 0
    while chunkstartmarker < xTrain.shape[0]:
        chunkrows = range(chunkstartmarker,chunkstartmarker+chunksize)
        X_chunk, y_chunk = getrows(chunkrows)
        yield X_chunk, y_chunk
        chunkstartmarker += chunksize
 
batcherator = iter_minibatches(chunksize=10000)
model = linear_model.SGDClassifier()
#model = LogisticRegression()
 
# Train model
for X_chunk, y_chunk in batcherator:
    model.partial_fit(X_chunk, y_chunk, classes=np.unique(yTrain))
 
# Now make predictions with trained model
y_predicted = model.predict(xTest)

c:\users\ratul\anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [21]:
dfObj = pd.DataFrame(y_predicted.tolist(), columns = ['label'])

In [22]:
li = []
for i in range(200000):
    li.append('test_' + str(i))

In [23]:
dfObj['ID_code'] = li

In [24]:
dfObj.head()

,label,ID_code
0,0,test_0
1,0,test_1
2,0,test_2
3,1,test_3
4,0,test_4


In [20]:
dfObj.to_csv('predicted.csv', index = None, header=True)